# A tutorial for 

# *PersLay: A Simple and Versatile Neural Network Layer for Persistence Diagrams*. 

__Author:__ Theo Lacombe

__Note:__ This is an alpha version of PersLay. Do not hesitate to contact the authors for any comment, suggestion, bug, etc.

# Introduction

Printing the current version of Python.

In [ ]:
import sys
print("Current version of your system: (we recommand Python 3.6)")
print(sys.version)

## Imports

In [ ]:
from utils import generate, visualization, load
from archi import perslay, baseModel
from preprocessing import preprocess
from expe import single_run

## Outline:
In this notebook:
- First, we select a dataset. Two types of datasets are provided by default, either synthetic orbits from dynamical systems, or real-life graph dataset. 
- Then, we generate the persistence diagrams (and other useful informations such as labels, etc.) for the chosen dataset.
- (Optional) we propose to visualize the generated diagrams.
- We define a neural network that uses some PersLay channels as first layers to handle persistence diagrams. This can be used as a guideline to use PersLay in your own experiments.
- We show how to train this neural network on the chosen dataset.
- Finally, we explain how you could use PersLay with your own persistence diagrams

# 1. Building diagrams and eventual features from a provided dataset

__Note:__ If you want to use PersLay with your own diagrams, skip this section and jump to 1bis.

We start by choosing the dataset we want to run the experiments on.

We suggest the user to start with `"MUTAG"` as this dataset is reasonably small (188 graphs with 18 nodes on average). Note that its small size implies a large variability in tests.

Available options are:

- Orbit datasets: `"ORBIT5K"`, `"ORBIT100K"`.

- Graphs datasets: `"MUTAG"`, `"BZR"`, `"COX2"`, `"DHFR"`, `"PROTEINS"`, `"NCI1"`, `"NCI109"`, `"FRANKENSTEIN"`,  `"IMDB-BINARY"`, `"IMDB-MULTI"`.

__Important note:__ `COLLAB`, `"REDDIT5K"` and `"REDDIT12K"` are not available yet (see README.md). Contact the authors for more information.

Beware that for the larger datasets (`COLLAB`, `REDDIT5K, REDDIT12K, ORBIT100K`), the needed files can be quite large (e.g. 3Gb for for `ORBIT100K`), so that RAM can be limiting, and time to generate the diagrams and running the experiments can be quite involving depending on the hardware available. You can have access to a description of the dataset in the Section B in the supplementary material of the article.

In [ ]:
# Chose your config file using one of the filename mentioned above.
dataset = "MUTAG"

Here, we implicitely load our data (saved as `.mat` files for graphs datasets, and generated on-the-fly for orbits datasets---which can take some time for `ORBIT100K` especially), and then compute the persistence diagrams that will be used in the classification experiment (requires to have `gudhi` installed). For graph datasets, we also generate a series of additional features (see [1]).

Running `generate` will store diagrams, features and labels. Therefore, it is sufficient to run it just once (for each different dataset).

Note that for bigger datasets, the computations of these diagrams can be quite long.

In [ ]:
generate(dataset)

Now we load and preprocess diagrams (to make them PersLay-compatible) and other useful items using the files that we have generated.

In [ ]:
feats, diags_tmp, filts, labels = load(dataset)

### Visualization (optional)

In [ ]:
# Run this cell to visualise some example of diagrams generated.
# Requires matplotlib.
visualization(diags_tmp, filts)

Now we preprocess our diagrams to make them PersLay-compatible.

In [ ]:
diags = preprocess(diags_tmp)

# 1bis. Alternative: use your own diagrams

__Note:__ Skip this section if you want to use the diagrams generated above.

We provide a (hopefully) convenient way to use your own diagrams for a classification task (with some eventual features).

Diagrams must be given in the following format:
Assume you have $N$ observations. For each of them, you build $K$ different diagrams (e.g. diagrams in different homology dimensions, for different filtration, etc.). 

Then, you must provide a `diags_tmp` variable that is a `dictionnary`, whose $K$ keys are the diagram type names (e.g. `Rips_dim_0`, `Cech_dim_1`). For each key $k_i$, $1 \leq i \leq K$, the corresponding value is a `list` of `np.arrays`, each array encoding a diagram. 

Note that each lists must have the same length $N$ (you need to have the same number of diagrams generated for each list). Note also that you must keep the order (i.e. the first element of each list must correspond to the diagrams generated with the first observation, and so on).

Below is an example of such a (very simple) dictionnary.

In [ ]:
# import numpy as np

# diags_tmp = {'Alpha0':[np.array([[0.1, 0.2], 
#                                 [0.2, 0.5],
#                                 [0.3, 0.9]]),  # a first dgm with 3 pts
#                       np.array([[0.1, 0.4], 
#                                 [0.3, 0.5]]),  # a second dgm with 2 pts
#                      ],  # end of the first diagram type
#            'Alpha1':[np.array([[0.1, 0.4], 
#                                 [0.2, 0.6],
#                                 [0.4, 0.9]]),
#                       np.array([[0.1, 0.2], 
#                                 [0.5, 0.7],
#                                 [0.8, 0.9]]),
#                      ]  # end of the second diagram type
#            }

In [ ]:
### To use your own diagrams, uncomment and complete the following
# diags_tmp = ...

Now, we apply a preprocessing that makes our sets of diagrams compatible with perslay.

In [ ]:
### Uncomment the following to process your diagrams (necessary)
# diags = preprocess(diags_tmp)

Now, you must (obviously) provide the labels corresponding to each diagrams (be careful that you keep the same order).

In [ ]:
### To use your own labels, uncomment and complete the following
# labels = ...

You can use some additional "standard" features in your network. These features must be provided as a $N \times d$ `np.array`, where $N$ is your number of observation (as before) and $d$ the dimension of your features.

If you do not want to use additional features, you must use an empty array of size $(N,0)$, where $N$ is the number of observations.

In [ ]:
### Uncomment and complete the following line to not use feat.
# N = ...   # number of observations
# feats = np.array([[]]*N)
### To use your own features instead, uncomment and complete the following
# feats = ...

# 2. Using PersLay in a neural network

We define a PersLay layer, and then a (very simple) neural network architecture that uses PersLay. This can be used as a template to build your own architecture using PersLay.

## 2.1 Set the hyper-parameters

Layer type, must be one of (see [1] for details):
- `"im"` for persistence image layer.
- `"pm"` for permutation equivarient layer (as in [2]).
- `"gs"` for a gaussian layer.
- `"ls"` for a landscape layer.

In [ ]:
layer_type = "im"

Choice of the weight function, either:
- `"linear"`
- `"grid"`, in this case the user must pick a grid size.

In [ ]:
weight = "grid"
grid_size = [10,10]  # only useful if weight=="grid"

Permutation invariant operator, must be one of:
- `"sum"`.
- `"topk"`, will select the $k$ highest values, specified in `keep`.

In [ ]:
perm_op = "sum"
keep = 5  # only useful if perm_op = "topk"

Now, there are some hyper parameters that are specific to different the layer types.

In [ ]:
# Parameter specific to layer_type="im"
image_size=[10, 10]
# Parameter specific to layer_type="gs"
num_gaussians=50
# Parameter specific to layer_type="pm"
d = 50  # Output dimension
# Parameter specific to layer_type="ls"
num_samples = 50

Now we concatenate all these parameters in a dictionnary that will be given to our model (see below).

In [ ]:
perslayParameters = {"layer_type":layer_type,
                     "perm_op": perm_op, "keep":keep,
                     "weight":weight, "grid_size":grid_size,
                     "image_size": image_size,
                     "num_gaussians": num_gaussians,
                     "pm_dimension": d,
                     "num_samples": num_samples}

__Note:__ There are some other parameters available to tune PersLay that are not detailed here. This will be updated later. Feel free to check the implementation provided in `archi.py`.

## 2.2 Designing the network

In the template below, we define a very simple `baseModel` that encodes a network architecture. In this model, we define a PersLay layer for each type of diagrams used in input, but all these layers have the same hyper-parameters (as in [1]).

Eventual additional features are simply concatenated with the output of these perslay layers, and a fully connected layer is then used to make the prediction.

In [ ]:
model = baseModel(perslayParameters, filts, labels)

## 2.3 Training the network

We can now train our network.

As any neural-network framework, PersLay benefits from the use of GPU(s). If a GPU is available (and `tensorflow-gpu` is installed), the computations should hopefully use it. Otherwise, the computations will be run on the cpu.

### Running the experiments.

We suggest the user to run a single-run first with the `single_run` function, that is training the network once and observing the performance (classification accuracy) on the test set.
- For orbit datasets, we suggest to use a 70-30 train-test split (to be consistent with [LY18]), i.e. `test_size = 0.3`.
- For graph datasets, we suggest to use a 90-10 train-test split (to be consistent with [ZWX+18]), i.e. `test_size = 0.1`.

In [ ]:
test_size = 0.1

The `single_run` function will load (and print) the network parameters as described in Table 5<CHECK LABEL>: perslay hyperparameters (choice of $\phi$, $w$...), optimizer (number of epochs, learning rate...), etc.
   
It then uses the diagrams (and eventual features) that have been generated when calling `generate(dataset)`, randomly split them into train/test sets, and use them to feed to the network. 

Train and Test accuracies are printed every 10 epochs during the training.

Note that (especially on small datasets like `MUTAG, COX2` etc.), there can be an important variability in the accuracy reported on different calls of `single_run`.

In [ ]:
# Specify optimization parameters.
decay = 0.9
lr = 0.01
num_epoch = 100
optim_parameters = {"decay":decay,"lr":lr, "num_epoch":num_epoch}

In [ ]:
single_run(diags, feats, labels, filts, model, optim_parameters, test_size)

# Bibliography

[1] _PersLay: A Simple and Versatile Neural Network Layer for Persistence Diagrams._
Mathieu Carrière, Frederic Chazal, Yuichi Ike, Théo Lacombe, Martin Royer, Yuhei Umeda.

[2] _Deep Sets._
Manzil Zaheer, Satwik Kottur, Siamak Ravanbakhsh, Barnabas Poczos, Ruslan R. Salakhutdinov, Alexander J. Smola.
_Advances in Neural Information Processing Systems 30 (NIPS 2017)_